In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# import cv2
# import os

# def video_to_frames(video_path, output_folder):
#     # Create the output directory if it does not exist
#     if not os.path.exists(output_folder):
#         os.makedirs(output_folder)

#     # Open the video file
#     vidcap = cv2.VideoCapture(video_path)
#     success, image = vidcap.read()
#     frame_count = 0

#     # Read frames until the video ends
#     while success:
#         # Save frame as a .jpg file
#         frame_name = os.path.join(output_folder, f"frame{frame_count:05d}.jpg")
#         cv2.imwrite(frame_name, image)

#         # Read the next frame
#         success, image = vidcap.read()
#         frame_count += 1

#     vidcap.release()
#     print(f"Extracted {frame_count} frames to {output_folder}")

# # Usage
# video_to_frames("/content/drive/MyDrive/Capstone/Datasets/Brazil_vs_Belgium_Cleaned.mp4", "/content/drive/MyDrive/output_frames_folder")


Extracted 175237 frames to /content/drive/MyDrive/output_frames_folder


In [ ]:
!unzip /content/drive/MyDrive/data.zip

Archive:  /content/drive/MyDrive/data.zip
  inflating: data/.DS_Store          
  inflating: data/demo.png           
  inflating: data/input.png          
  inflating: data/output.png         
 extracting: data/score.png          
 extracting: data/team1.png          
 extracting: data/team2.png          
 extracting: data/time.png           


In [ ]:
!unzip /content/drive/MyDrive/model.zip

Archive:  /content/drive/MyDrive/model.zip
  inflating: model/crnn.pth          


In [ ]:
import cv2
import torch
import random
import collections
import numpy as np
from PIL import Image
import torch.nn as nn
import matplotlib.pyplot as plt
from torch.autograd import Variable
import torchvision.transforms as transforms

In [ ]:
class strLabelConverter(object):

    def __init__(self, alphabet):

        alphabet = alphabet.lower()
        self.alphabet = alphabet + '-'

        self.dict = {}

        for i, char in enumerate(alphabet):

            self.dict[char] = i + 1      # Position 0 is for space character

    def encode(self, text):

        # Encoding the word into integer format.
        text = [self.dict[char.lower()]for char in text]
        length = [len(text)]
        return (torch.IntTensor(text), torch.IntTensor(length))

    def decode(self, t, length, raw=False):

        length = length[0]
        assert t.numel() == length, "text with length: {} does not match declared length: {}".format(t.numel(), length)
        if raw:
            return ''.join([self.alphabet[i - 1] for i in t])
        else:
            char_list = []
            for i in range(length):
                if t[i] != 0 and (not (i > 0 and t[i - 1] == t[i])):
                    char_list.append(self.alphabet[t[i] - 1])
            return ''.join(char_list)

In [ ]:
class BidirectionalLSTM(nn.Module):

    def __init__(self, nIn, nHidden, nOut):
        super(BidirectionalLSTM, self).__init__()

        self.rnn = nn.LSTM(nIn, nHidden, bidirectional=True)
        self.embedding = nn.Linear(nHidden * 2, nOut)

    def forward(self, input):
        recurrent, _ = self.rnn(input)
        T, b, h = recurrent.size()
        t_rec = recurrent.view(T * b, h)

        output = self.embedding(t_rec)  # [T * b, nOut]
        output = output.view(T, b, -1)

        return output


class CRNN(nn.Module):

    def __init__(self):

        super(CRNN, self).__init__()

        cnn = nn.Sequential()
        cnn.add_module('conv0',      nn.Conv2d(1, 64, 3, 1, 1))  # Input, Output, Kernal, Stride, Padding
        cnn.add_module('relu0',      nn.ReLU(True))
        cnn.add_module('pooling0',   nn.MaxPool2d(2, 2))                     # 64x16x64
        cnn.add_module('conv1',      nn.Conv2d(64, 128, 3, 1, 1))
        cnn.add_module('relu1',      nn.ReLU(True))
        cnn.add_module('pooling1',   nn.MaxPool2d(2, 2))                     # 128x8x32
        cnn.add_module('conv2',      nn.Conv2d(128, 256, 3, 1, 1))
        cnn.add_module('batchnorm2', nn.BatchNorm2d(256))
        cnn.add_module('relu2',      nn.ReLU(True))
        cnn.add_module('conv3',      nn.Conv2d(256, 256, 3, 1, 1))
        cnn.add_module('relu3',      nn.ReLU(True))
        cnn.add_module('pooling2',   nn.MaxPool2d((2, 2), (2, 1), (0, 1)))   # 256x4x16
        cnn.add_module('conv4',      nn.Conv2d(256, 512, 3, 1, 1))
        cnn.add_module('batchnorm4', nn.BatchNorm2d(512))
        cnn.add_module('relu4',      nn.ReLU(True))
        cnn.add_module('conv5',      nn.Conv2d(512, 512, 3, 1, 1))
        cnn.add_module('relu5',      nn.ReLU(True))
        cnn.add_module('pooling3',   nn.MaxPool2d((2, 2), (2, 1), (0, 1)))   # 512x2x16
        cnn.add_module('conv6',      nn.Conv2d(512, 512, 2, 1, 0))           # 512x1x16
        cnn.add_module('batchnorm6', nn.BatchNorm2d(512))
        cnn.add_module('relu6',      nn.ReLU(True))

        self.cnn = cnn
        self.rnn = nn.Sequential(
            BidirectionalLSTM(512, 256, 256),       # Input, Hidden, Output
            BidirectionalLSTM(256, 256, 37))        # Final output: 37 classes

    def forward(self, input):
        # conv features
        conv = self.cnn(input)
        b, c, h, w = conv.size()
        assert h == 1, "the height of conv must be 1"
        conv = conv.squeeze(2)
        conv = conv.permute(2, 0, 1)  # [w, b, c]
        # rnn features
        output = self.rnn(conv)

        return output

In [ ]:
class resizeNormalize(object):

    def __init__(self, size, interpolation=Image.BILINEAR):
        self.size = size
        self.interpolation = interpolation
        self.toTensor = transforms.ToTensor()

    def __call__(self, img):
        img = img.resize(self.size, self.interpolation)
        img = self.toTensor(img)
        img.sub_(0.5).div_(0.5)
        return img

In [ ]:
import torch
from torch.autograd import Variable
import cv2
import matplotlib.pyplot as plt
from PIL import Image
import os

# Initialize model, transformer, and other necessary components
def initialize_model():
    model_path = './model/crnn.pth'
    alphabet = '0123456789abcdefghijklmnopqrstuvwxyz'
    model = CRNN()

    if torch.cuda.is_available():
        model = model.cuda()
    model.load_state_dict(torch.load(model_path))
    converter = strLabelConverter(alphabet)
    transformer = resizeNormalize((100, 32))
    return model, converter, transformer

# Prediction function
def predict(model, converter, transformer, image_path):
    image = Image.open(image_path).convert('L')
    image = transformer(image)
    if torch.cuda.is_available():
        image = image.cuda()
    image = image.view(1, *image.size())
    image = Variable(image)

    model.eval()
    preds = model(image)

    _, preds = preds.max(2)
    preds = preds.transpose(1, 0).contiguous().view(-1)
    preds_size = Variable(torch.IntTensor([preds.size(0)]))
    sim_pred = converter.decode(preds.data, preds_size.data, raw=False)
    return sim_pred

# Load image and prepare the cropped sections for OCR
def load_image(name):
    image = cv2.imread("/content/drive/MyDrive/frames/" + name, cv2.IMREAD_GRAYSCALE)
    # print(image)
    image = cv2.resize(image, (1280, 720))
    return image

def crop(box, name, img):
    crop_img = img[box[0]:box[1], box[2]:box[3]]
    cv2.imwrite('data/' + name, crop_img)

# Define boxes for cropping
team1_box = [43, 70, 110, 170]
score_box = [43, 70, 180, 250]
team2_box = [43, 70, 260, 320]
time_box = [43, 70, 365, 430]

# Main function to process multiple images
def process_images(image_folder):
    model, converter, transformer = initialize_model()
    previous_score=None
    # previous_time=None


    for idx, image_name in enumerate(sorted(os.listdir(image_folder))):
        # print(image_name)
        image = load_image(image_name)
        # print(previous_score)
        crop(team1_box, "team1.png", image)
        crop(score_box, "score.png", image)
        crop(time_box, "time.png", image)
        crop(team2_box, "team2.png", image)

        time = predict(model, converter, transformer, "data/time.png")
        team1 = predict(model, converter, transformer, "data/team1.png")
        team2 = predict(model, converter, transformer, "data/team2.png")
        score = predict(model, converter, transformer, "data/score.png")
        # print(time)
        # print(len(time))
        # Formatting time and score as required
        if(len(time))==3:
          time=time[:1]+time[1:]
        if len(time) == 4 :
          time = time[:2] +":"+ time[2:]
        if len(time)==5:
          time = time[:2] + ":" + time[3:]
        if len(time) == 3:
          time = time[:1] + ":" + time[1:]

        # if len(time) == 5:
        #     time = time[:2] + time[3:]
        # time = time[:2] + ":" + time[2:]

        if len(score) == 3:
          score = score[:1] + score[2:]

        if score[:1].isdigit() and score[1:].isdigit():
          score = score[:1] + "-" + score[1:]
        else:
          score = "!-!"
        # print("Image:", image_name)
        # print("Time:       ", time)
        # print("Team 1:     ", team1)
        # print("Team 2:     ", team2)
        # print("Score:      ", score)
        # print("#############")
        # print(len(score))
        # Print the first frame details or whenever there's a change in the score/time
        if(previous_score==None):
            print("Image:", image_name)
            print("Time:       ", time)
            print("Team 1:     ", team1)
            print("Team 2:     ", team2)
            print("Score:      ", score)

        elif (score != previous_score and (score!="!-!" and len(score)==3 )):

            print("Image:", image_name)
            print("Time:       ", time)
            print("Team 1:     ", team1)
            print("Team 2:     ", team2)
            print("Score:      ", score)
        if score!="!-!":
          previous_score = score

        # Update previous score and time


# Folder containing all images
process_images("/content/drive/MyDrive/frames")


<ipython-input-9-f86d2a50f2d0>:16: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path))


Image: frame00052.jpg
Time:        00:01
Team 1:      bra
Team 2:      bel
Score:       0-0
Image: frame23378.jpg
Time:        12:59
Team 1:      bra
Team 2:      bel
Score:       0-1
Image: frame55636.jpg
Time:        30:54
Team 1:      bra
Team 2:      bel
Score:       0-2
Image: frame_138840.jpg
Time:        75:27
Team 1:      bra
Team 2:      bel
Score:       1-2


In [ ]:
import os
import cv2

def load_image(name):
    # Construct the full path to the image
    image_path = os.path.join("output_frames_folder", name)
    print(f"Attempting to load image: {image_path}")  # Debug print

    # Read the image in grayscale
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

    # Check if the image was loaded successfully
    if image is None:
        print(f"Error: Image {name} not found or unable to open.")
        return None

    # Resize the image if it loaded correctly
    image = cv2.resize(image, (1280, 720))
    return image

def process_images(folder_path):
    # List all files in the folder
    files = os.listdir(folder_path)
    print(f"Files in '{folder_path}': {files}")  # Debug print

    # Process only files ending with .jpg
    for filename in files:
        if filename.endswith(".jpg"):
            image = load_image(filename)
            if image is not None:
                # Process the image
                print(f"Processed {filename}")
                break  # Break after processing the first image to limit output

# Call the function with the path to your folder containing frames
process_images("/content/drive/MyDrive/output_frames_folder")


Streaming output truncated to the last 5000 lines.
Error: Image frame60219.jpg not found or unable to open.
Attempting to load image: output_frames_folder/frame60220.jpg
Error: Image frame60220.jpg not found or unable to open.
Attempting to load image: output_frames_folder/frame60221.jpg
Error: Image frame60221.jpg not found or unable to open.
Attempting to load image: output_frames_folder/frame60222.jpg
Error: Image frame60222.jpg not found or unable to open.
Attempting to load image: output_frames_folder/frame60223.jpg
Error: Image frame60223.jpg not found or unable to open.
Attempting to load image: output_frames_folder/frame60224.jpg
Error: Image frame60224.jpg not found or unable to open.
Attempting to load image: output_frames_folder/frame60225.jpg
Error: Image frame60225.jpg not found or unable to open.
Attempting to load image: output_frames_folder/frame60226.jpg
Error: Image frame60226.jpg not found or unable to open.
Attempting to load image: output_frames_folder/frame60227.j

KeyboardInterrupt: 

In [ ]:
#######################################################################################################################################################
BELOW IS THE SAME PROCESS FOR DIFFERENT VIDEO

In [ ]:
import torch
from torch.autograd import Variable
import cv2
import matplotlib.pyplot as plt
from PIL import Image
import os

# Initialize model, transformer, and other necessary components
def initialize_model():
    model_path = './model/crnn.pth'
    alphabet = '0123456789abcdefghijklmnopqrstuvwxyz'
    model = CRNN()

    if torch.cuda.is_available():
        model = model.cuda()
    model.load_state_dict(torch.load(model_path))
    converter = strLabelConverter(alphabet)
    transformer = resizeNormalize((100, 32))
    return model, converter, transformer

# Prediction function
def predict(model, converter, transformer, image_path):
    image = Image.open(image_path).convert('L')
    image = transformer(image)
    if torch.cuda.is_available():
        image = image.cuda()
    image = image.view(1, *image.size())
    image = Variable(image)

    model.eval()
    preds = model(image)

    _, preds = preds.max(2)
    preds = preds.transpose(1, 0).contiguous().view(-1)
    preds_size = Variable(torch.IntTensor([preds.size(0)]))
    sim_pred = converter.decode(preds.data, preds_size.data, raw=False)
    return sim_pred

# Load image and prepare the cropped sections for OCR
def load_image(name):
    image = cv2.imread("/content/drive/MyDrive/frames/" + name, cv2.IMREAD_GRAYSCALE)
    # print(image)
    image = cv2.resize(image, (1280, 720))
    return image

def crop(box, name, img):
    crop_img = img[box[0]:box[1], box[2]:box[3]]
    cv2.imwrite('data/' + name, crop_img)

# Define boxes for cropping
team1_box = [43, 70, 110, 170]
score_box = [43, 70, 180, 250]
team2_box = [43, 70, 260, 320]
time_box = [43, 70, 365, 430]

# Main function to process multiple images
def process_images(image_folder):
    model, converter, transformer = initialize_model()
    previous_score=None
    # previous_time=None


    for idx, image_name in enumerate(sorted(os.listdir(image_folder))):
        # print(image_name)
        image = load_image(image_name)
        # print(previous_score)
        crop(team1_box, "team1.png", image)
        crop(score_box, "score.png", image)
        crop(time_box, "time.png", image)
        crop(team2_box, "team2.png", image)

        time = predict(model, converter, transformer, "data/time.png")
        team1 = predict(model, converter, transformer, "data/team1.png")
        team2 = predict(model, converter, transformer, "data/team2.png")
        score = predict(model, converter, transformer, "data/score.png")
        # print(time)
        # print(len(time))
        # Formatting time and score as required
        if(len(time))==3:
          time=time[:1]+time[1:]
        if len(time) == 4 :
          time = time[:2] +":"+ time[2:]
        if len(time)==5:
          time = time[:2] + ":" + time[3:]
        if len(time) == 3:
          time = time[:1] + ":" + time[1:]

        # if len(time) == 5:
        #     time = time[:2] + time[3:]
        # time = time[:2] + ":" + time[2:]

        if len(score) == 3:
          score = score[:1] + score[2:]

        if score[:1].isdigit() and score[1:].isdigit():
          score = score[:1] + "-" + score[1:]
        else:
          score = "!-!"
        # print("Image:", image_name)
        # print("Time:       ", time)
        # print("Team 1:     ", team1)
        # print("Team 2:     ", team2)
        # print("Score:      ", score)
        # print("#############")
        # print(len(score))
        # Print the first frame details or whenever there's a change in the score/time
        if(previous_score==None):
            print("Image:", image_name)
            print("Time:       ", time)
            print("Team 1:     ", team1)
            print("Team 2:     ", team2)
            print("Score:      ", score)

        elif (score != previous_score and (score!="!-!" and len(score)==3 )):

            print("Image:", image_name)
            print("Time:       ", time)
            print("Team 1:     ", team1)
            print("Team 2:     ", team2)
            print("Score:      ", score)
        if score!="!-!":
          previous_score = score

        # Update previous score and time


# Folder containing all images
process_images("/content/drive/MyDrive/frames")
